In [7]:
import pandas as pd
import numpy as np
import tushare as ts
import datetime
import time, datetime

In [8]:
#连接tushare
pro = ts.pro_api('31e6c92e6133d112d8ee557bb245044fde5daab04104598620f23921')
df = df = pro.index_daily(ts_code='399300.SZ', start_date='20200720', end_date='20200721')

In [9]:
df

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,399300.SZ,20200721,4691.0425,4697.5026,4714.2925,4661.4440,4680.3046,10.7379,0.2294,200415659.0,356069805.7
1,399300.SZ,20200720,4680.3046,4597.2038,4681.9431,4534.2305,4544.7007,135.6039,2.9838,246068914.0,416714222.2


In [18]:
df[['high','trade_date']]

,high,trade_date
0,6.53,20200721
1,6.48,20200720


In [11]:
dfg = df = pro.daily(ts_code='600366.SH', start_date='20200720', end_date='20200721')
dfg

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600366.SH,20200721,6.43,6.53,6.36,6.44,6.44,0.00,0.000,234510.99,150748.756
1,600366.SH,20200720,6.13,6.48,6.13,6.44,6.11,0.33,5.401,367330.69,233637.510


In [15]:
ff = dfg['close1']
dfg.drop(columns='close1',inplace=True)
dfg.insert(2,'close',ff)

In [16]:
dfg

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,600366.SH,20200721,6.44,6.43,6.53,6.36,6.44,0.00,0.000,234510.99,150748.756
1,600366.SH,20200720,6.44,6.13,6.48,6.13,6.11,0.33,5.401,367330.69,233637.510


In [49]:
class STOCK():

    _prefix_url = "https://hq.sinajs.cn/list="

    #中国指数
    def index(self, code):
        exchange = "sz" if (code[:3] == "399") else "sh"
        url = self._prefix_url + exchange + code       #http://hq.sinajs.cn/list=s_sh000001
        info = self.http_request(url)
        result = self.data_format(info)
        return pd.DataFrame(result,index=[0])

    #美股，港股指数
    def index_world(self, ex_code):
        url = self._prefix_url + ex_code       #http://hq.sinajs.cn/list=s_sh000001
        info = self.http_request(url)
        result = self.data_format(info)
        return pd.DataFrame(result,index=[0])

    #中国个股
    def individual(self, code):
        if ((code[:3] == "600") | (code[:3] == "900") | (code[:3] == "688")):   #上证A股600，上证B股900，科创板688
            exchange = "sh"
        elif ((code[:2] == "43") | (code[:2] == "83") | (code[:2] == "87")): #新三板43和83，87
            exchange = "sb"
        elif ((code[:3] == "000") | (code[:3] == "200")): #深证A股000，深证B股200
            exchange = "sz"
        elif ((code[:3] == "002")): #中小板002
            exchange = "sz"
        elif ((code[:3] == "300")): #创业板300
            exchange = "sz"
        # else:
            # exchange = "sz"        #其他默认sz
        url = self._prefix_url + exchange + code
        info = self.http_request(url)
        result = self.data_format(info)
        return pd.DataFrame(result,index=[0])
    #数据标准化函数
    def data_format(self, data):
        rawdata = re.search('(")(.+)(")', data).group(2)
        stockdata = rawdata.split(",")
        if ((stockdata[0] == "道琼斯") | (stockdata[0] == "纳斯达克") | (stockdata[0] == "标普500指数")):
            ts = int(time.mktime(time.strptime(stockdata[3], "%Y-%m-%d %H:%M:%S")))
            tss=time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(ts))#转换成标准日期
            ttss=pd.to_datetime(tss,format='%Y-%m-%d').date()
            dict = {
                    'time': ttss,
                    'open': float(stockdata[5]),
                    'high': float(stockdata[6]),
                    'low': float(stockdata[7]),
                    'close': float(stockdata[1]),
                    'amount': None,
                    'volume': float(stockdata[10]), #成交量
                    'code': code,
                    'name': stockdata[0]
                    
                    }

        elif (stockdata[1] == "恒生指数"):
            ts = int(time.mktime(time.strptime('%s %s'%(stockdata[17],stockdata[18]) , "%Y/%m/%d %H:%M:%S")))
            tss=time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(ts))#转换成标准日期
            ttss=pd.to_datetime(tss,format='%Y-%m-%d').date()
            dict = {'time': ttss,  
                    'open': float(stockdata[2]),
                    'high': float(stockdata[4]),
                    'low': float(stockdata[5]),
                    'close': float(stockdata[6]),
                    'amount': float(stockdata[11]),       #成交额
                    'volume': None,       #成交量
                    'code': code,
                    'name': stockdata[1]
                    }
        else:
            ts = int(time.mktime(time.strptime('%s %s'%(stockdata[30],stockdata[31]) , "%Y-%m-%d %H:%M:%S")))
            tss=time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(ts))#转换成标准日期
            ttss=pd.to_datetime(tss,format='%Y-%m-%d').date()
            dict = {'time': ttss,                    
                    'open': float(stockdata[1]),
                    'high': float(stockdata[4]),
                    'low': float(stockdata[5]),
                    'close': float(stockdata[3]),
                    'amount': float(stockdata[9]),       #成交额
                    'volume': float(stockdata[8]),       #成交量
                    'code': code,
                    'name': stockdata[0]
                    }
        return dict

    # HTTP请求
    def http_request(self, url):
        try:
            req = urllib.request.Request(url)
            req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36')
            r = urllib.request.urlopen(req)
            result = r.read().decode('gb2312')
            return result
        except Exception as e:
            print([url, repr(e)])

In [50]:
if __name__ == "__main__":
    result =[]
    sql="insert into sz_daily (date_id,open1,high,low,close,amount,vol,code_id) values(%s,%s,%s,%s,%s,%s,%s,%s)"
    stock = STOCK() #创建类对象
    #遍历国内主要指数
    chinaindexes = [
        "000001", # sh000001 上证指数
        "399001", # sz399001 深证成指
        "000300", # sh000300 沪深300
        "399005", # sz399005 中小板指
        "399006", # sz399006 创业板指
        
    ]
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123', db='tdx')
    cursor = conn.cursor()  #连接数据库  
    for code in chinaindexes:
        result.append(stock.index(code))
 

In [51]:
type(result)

list

In [52]:
result[1]


,time,open,high,low,close,amount,volume,code,name
0,2020-07-20,13303.803,13448.85,13032.396,13448.85,6.490005e+11,4.935472e+10,399001,深证成指


In [ ]:
        result['time']=pd.to_datetime(result['time'],format='%Y-%m-%d').date()
        #cursor.execute(sql,dl)# 每日的数据添加一次，不再汇总后添加
        #conn.commit() 
    #cursor.close()     # 关闭游标
    #conn.close()      # 关闭连接

In [ ]:
    #遍历国内股票
    chinastockes = [
        "000063",   #中兴通讯
        "688029",   #科创板
    ]
    for code in chinastockes:
        result = stock.individual(code)
        print(result)

In [ ]:
    #美股，港股指数
    # https://hq.sinajs.cn/list=int_dji,int_nasdaq,int_sp500,int_hangseng 简版
    # https://hq.sinajs.cn/list=gb_dji,gb_ixic,gb_inx,rt_hkHSI              复杂版
    worldstockes = [
        "gb_dji",   #  道琼斯
        "gb_ixic",  #  纳斯达克
        "gb_inx",   #  标普指数
        "rt_hkHSI", #  恒生指数
    ]
    for code in worldstockes:
        result = stock.index_world(code)
        print(result)